$\textbf{Intro}$

In this work, we will build a 5-Day Mean Reversion alpha factor on a portfolio of 10 stocks using 10 years historical data, and evaluate it on different hodling periods. We denote this portfolio the universe as our trading strategy is built and statistically appreciated within this dataset. Portfolio and universe will be used interchangeably.

The selected stocks stand as the most traded stocks in average during the ten years considered; they also possed at least 8 years of trading data. 

The evaluation task is done with the alphalens library. 

Evaluation metrics include, the forward returns and their Sharpe Ratio, the information coefficient, Turnover, etc.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
idx = pd.IndexSlice

In [3]:
data = pd.read_csv('WIKI_PRICES.csv', parse_dates=True, index_col=[0,1])

In [5]:
data

open   high      low  close      volume  ex-dividend  \
ticker date                                                                
A      1999-11-18  45.50  50.00  40.0000  44.00  44739900.0          0.0   
       1999-11-19  42.94  43.00  39.8100  40.38  10897100.0          0.0   
       1999-11-22  41.31  44.00  40.0600  44.00   4705200.0          0.0   
       1999-11-23  42.50  43.63  40.2500  40.25   4274400.0          0.0   
       1999-11-24  40.13  41.94  40.0000  41.06   3464400.0          0.0   
...                  ...    ...      ...    ...         ...          ...   
ZUMZ   2018-03-21  23.80  24.60  23.6058  23.95    354092.0          0.0   
       2018-03-22  23.90  24.35  23.3000  23.35    269607.0          0.0   
       2018-03-23  23.55  24.20  23.4500  23.55    301584.0          0.0   
       2018-03-26  23.75  24.80  23.7000  24.65    375320.0          0.0   
       2018-03-27  24.65  24.65  23.3500  23.60    403884.0          0.0   

                   split_ratio   adj_open   adj_high    adj_low  adj_close  \
ticker date                                                                  
A      1999-11-18          1.0  31.041951  34.112034  27.289627  30.018590   
       1999-11-19          1.0  29.295415  29.336350  27.160002  27.548879   
       1999-11-22          1.0  28.183363  30.018590  27.330562  30.018590   
       1999-11-23          1.0  28.995229  29.766161  27.460188  27.460188   
       1999-11-24          1.0  27.378319  28.613174  27.289627  28.012803   
...                        ...        ...        ...        ...        ...   
ZUMZ   2018-03-21          1.0  23.800000  24.600000  23.605800  23.950000   
       2018-03-22          1.0  23.900000  24.350000  23.300000  23.350000   
       2018-03-23          1.0  23.550000  24.200000  23.450000  23.550000   
       2018-03-26          1.0  23.750000  24.800000  23.700000  24.650000   
       2018-03-27          1.0  24.650000  24.650000  23.350000  23.600000   

                   adj_volume  
ticker date                    
A      1999-11-18  44739900.0  
       1999-11-19  10897100.0  
       1999-11-22   4705200.0  
       1999-11-23   4274400.0  
       1999-11-24   3464400.0  
...                       ...  
ZUMZ   2018-03-21    354092.0  
       2018-03-22    269607.0  
       2018-03-23    301584.0  
       2018-03-26    375320.0  
       2018-03-27    403884.0  

[15389314 rows x 12 columns]

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15389314 entries, ('A', Timestamp('1999-11-18 00:00:00')) to ('ZUMZ', Timestamp('2018-03-27 00:00:00'))
Data columns (total 12 columns):
 #   Column       Dtype  
---  ------       -----  
 0   open         float64
 1   high         float64
 2   low          float64
 3   close        float64
 4   volume       float64
 5   ex-dividend  float64
 6   split_ratio  float64
 7   adj_open     float64
 8   adj_high     float64
 9   adj_low      float64
 10  adj_close    float64
 11  adj_volume   float64
dtypes: float64(12)
memory usage: 1.4+ GB


 We begin the selection process of our universe. Note that the attributes such as  adjacent open, adjacent high, adjacent low, adjacent close and adjacent volume are considered. 

In [29]:
data = data.filter(like='adj_').rename(columns=lambda x: x.replace('adj_', '')).loc[idx[:, '2006':'2016'], :].dropna()

In [30]:
data

open       high        low      close     volume
ticker date                                                             
A      2006-01-03  22.786839  22.909642  22.387796  22.855063  3796200.0
       2006-01-04  22.889175  23.080202  22.766372  22.916465  3001300.0
       2006-01-05  22.820951  23.516836  22.820951  23.516836  3458800.0
       2006-01-06  23.537304  23.735153  23.257585  23.639640  4396500.0
       2006-01-09  23.639640  23.741976  23.482724  23.571416  2920500.0
...                      ...        ...        ...        ...        ...
ZUMZ   2016-12-23  20.950000  21.500000  20.950000  21.350000   532292.0
       2016-12-27  21.200000  21.700000  21.200000  21.450000   308004.0
       2016-12-28  21.550000  21.749900  21.325000  21.450000   165827.0
       2016-12-29  21.550000  22.050000  21.400000  21.900000   322108.0
       2016-12-30  21.900000  22.190000  21.600000  21.850000   295429.0

[7531907 rows x 5 columns]

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7531907 entries, ('A', Timestamp('2006-01-03 00:00:00')) to ('ZUMZ', Timestamp('2016-12-30 00:00:00'))
Data columns (total 5 columns):
 #   Column  Dtype  
---  ------  -----  
 0   open    float64
 1   high    float64
 2   low     float64
 3   close   float64
 4   volume  float64
dtypes: float64(5)
memory usage: 316.8+ MB


The goal is to build the universe on which alpha factors will be evaluated.

The universe consists of 10 most traded stocks which have trading data of at least 8 years. 

In [32]:
dv = data.close.mul(data.volume)
dv

ticker  date      
A       2006-01-03    8.676239e+07
        2006-01-04    6.877919e+07
        2006-01-05    8.134003e+07
        2006-01-06    1.039317e+08
        2006-01-09    6.884032e+07
                          ...     
ZUMZ    2016-12-23    1.136443e+07
        2016-12-27    6.606686e+06
        2016-12-28    3.556989e+06
        2016-12-29    7.054165e+06
        2016-12-30    6.455124e+06
Length: 7531907, dtype: float64

In [33]:
dv = dv.unstack('ticker').dropna(thresh=8*252, axis=1).rank(axis=1, ascending=True).mean(axis=0).nlargest(10)
dv

ticker
AAPL     2449.400506
GOOGL    2445.394727
MSFT     2443.527266
BAC      2442.746479
XOM      2441.949079
C        2440.485374
JPM      2435.513904
GE       2434.914048
INTC     2434.508126
CSCO     2431.022030
dtype: float64

The resulting universe of stocks that is:

In [35]:
dv.index.to_list()

['AAPL', 'GOOGL', 'MSFT', 'BAC', 'XOM', 'C', 'JPM', 'GE', 'INTC', 'CSCO']

In [36]:
# now the universe
universe = data.loc[idx[dv.index.to_list(), :], :]
universe

open       high        low      close       volume
ticker date                                                               
AAPL   2006-01-03   9.295670   9.606417   9.285132   9.606417  201808600.0
       2006-01-04   9.655252   9.764489   9.574288   9.634690  154900900.0
       2006-01-05   9.616698   9.625694   9.477903   9.558867  112355600.0
       2006-01-06   9.669389   9.857019   9.580714   9.805613  176114400.0
       2006-01-09   9.860874   9.921276   9.733646   9.773485  168760200.0
...                      ...        ...        ...        ...          ...
CSCO   2016-12-23  29.484966  29.542893  29.349802  29.475311    9167598.0
       2016-12-27  29.581511  29.755293  29.542893  29.620129   14175488.0
       2016-12-28  29.639438  29.707020  29.301529  29.369111   12022231.0
       2016-12-29  29.320838  29.494620  29.282220  29.407729   10995584.0
       2016-12-30  29.504275  29.542893  29.089130  29.176020   20190029.0

[27690 rows x 5 columns]

In [37]:
universe.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 27690 entries, ('AAPL', Timestamp('2006-01-03 00:00:00')) to ('CSCO', Timestamp('2016-12-30 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    27690 non-null  float64
 1   high    27690 non-null  float64
 2   low     27690 non-null  float64
 3   close   27690 non-null  float64
 4   volume  27690 non-null  float64
dtypes: float64(5)
memory usage: 1.9+ MB


In [38]:
universe.index.unique('ticker')

Index(['AAPL', 'GOOGL', 'MSFT', 'BAC', 'XOM', 'C', 'JPM', 'GE', 'INTC',
       'CSCO'],
      dtype='object', name='ticker')

In [39]:
# remove outliers of daily returns
daily_returns = universe.close.pct_change()
daily_returns

ticker  date      
AAPL    2006-01-03         NaN
        2006-01-04    0.002943
        2006-01-05   -0.007870
        2006-01-06    0.025813
        2006-01-09   -0.003277
                        ...   
CSCO    2016-12-23    0.002298
        2016-12-27    0.004913
        2016-12-28   -0.008475
        2016-12-29    0.001315
        2016-12-30   -0.007879
Name: close, Length: 27690, dtype: float64

In [40]:
daily_returns.between(-1, +1)

ticker  date      
AAPL    2006-01-03    False
        2006-01-04     True
        2006-01-05     True
        2006-01-06     True
        2006-01-09     True
                      ...  
CSCO    2016-12-23     True
        2016-12-27     True
        2016-12-28     True
        2016-12-29     True
        2016-12-30     True
Name: close, Length: 27690, dtype: bool

In [41]:
universe = universe[daily_returns.between(-1, +1)]
universe

open       high        low      close       volume
ticker date                                                               
AAPL   2006-01-04   9.655252   9.764489   9.574288   9.634690  154900900.0
       2006-01-05   9.616698   9.625694   9.477903   9.558867  112355600.0
       2006-01-06   9.669389   9.857019   9.580714   9.805613  176114400.0
       2006-01-09   9.860874   9.921276   9.733646   9.773485  168760200.0
       2006-01-10   9.799188  10.524006   9.745212  10.391637  569967300.0
...                      ...        ...        ...        ...          ...
CSCO   2016-12-23  29.484966  29.542893  29.349802  29.475311    9167598.0
       2016-12-27  29.581511  29.755293  29.542893  29.620129   14175488.0
       2016-12-28  29.639438  29.707020  29.301529  29.369111   12022231.0
       2016-12-29  29.320838  29.494620  29.282220  29.407729   10995584.0
       2016-12-30  29.504275  29.542893  29.089130  29.176020   20190029.0

[27688 rows x 5 columns]

In [42]:
universe.index.unique('ticker')

Index(['AAPL', 'GOOGL', 'MSFT', 'BAC', 'XOM', 'C', 'JPM', 'GE', 'INTC',
       'CSCO'],
      dtype='object', name='ticker')

# Build and evaluate alpha factors performance

### build alpha factor: 5-Day mean-reversion

We build a 5-day mean-reversion alpha factor. We will later evaluate its performance on different holding periods using alphalens library

In [ ]:
# to be continued